# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.window import Window

C:\Users\Jakub.Pitera\AppData\Local\Temp\ipykernel_14872\3349515680.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Function definitions

In [2]:
def missing_values_count(df):
    return df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
# Start a Spark Session
spark = SparkSession.builder \
    .appName("InterviewChallenge") \
    .master("local[2]") \
    .getOrCreate()

#     .config("spark.executor.memory", "2g") \

In [4]:
# Read input data to spark dataframe
flights_path = 'Data/flights.csv' 
airlines_path = 'Data/airlines.csv'
airports_path = 'Data/airports.csv'

df = spark.read.options(header=True).csv(flights_path)  # main dataset with flights data
# df = spark.read.options(header=True, inferSchema=True).csv(flights_path)  # main dataset with flights data
airlines_df = spark.read.options(header=True, inferSchema=True).csv(airlines_path)  # data dict with airlines
airports_df = spark.read.options(header=True, inferSchema=True).csv(airports_path)  # data dict with airports

In [5]:
df.printSchema()

root
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: string (nullable = true)
 |-- ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: string (nullable = true)
 |-- D

In [6]:
# DTypes in pd formatting
pd.DataFrame(df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,YEAR,string
1,MONTH,string
2,DAY,string
3,DAY_OF_WEEK,string
4,AIRLINE,string
5,FLIGHT_NUMBER,string
6,TAIL_NUMBER,string
7,ORIGIN_AIRPORT,string
8,DESTINATION_AIRPORT,string
9,SCHEDULED_DEPARTURE,string


In [7]:
df.show(20)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [8]:
# Dataframe shape
print("Dataframe row count: ", df.count())
print("Dataframe columns count: ", len(df.columns))

Dataframe row count:  5819079
Dataframe columns count:  31


In [9]:
# Column statistics
# df.describe().toPandas()

In [10]:
missing_values_count(df)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

##### Airports data dictionary

In [11]:
airports_df.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)



In [12]:
# DTypes in pd formatting
pd.DataFrame(airports_df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,IATA_CODE,string
1,AIRPORT,string
2,CITY,string
3,STATE,string
4,COUNTRY,string
5,LATITUDE,double
6,LONGITUDE,double


In [13]:
airports_df.show(20)

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132|  -99.6819|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [14]:
# Dataframe shape
print("Dataframe row count: ", airports_df.count())
print("Dataframe columns count: ", len(airports_df.columns))

Dataframe row count:  322
Dataframe columns count:  7


In [15]:
# Column statistics
airports_df.describe().toPandas()

,summary,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,count,322,322,322,322,322,319,319
1,mean,None,None,None,None,None,38.981243918495295,-98.37896445141064
2,stddev,None,None,None,None,None,8.616735581018041,21.523492046498102
3,min,ABE,Aberdeen Regional Airport,Aberdeen,AK,USA,13.48345,-176.64603
4,max,YUM,Yuma International Airport,Yuma,WY,USA,71.28545,-64.79856


In [16]:
missing_values_count(airports_df)

+---------+-------+----+-----+-------+--------+---------+
|IATA_CODE|AIRPORT|CITY|STATE|COUNTRY|LATITUDE|LONGITUDE|
+---------+-------+----+-----+-------+--------+---------+
|        0|      0|   0|    0|      0|       3|        3|
+---------+-------+----+-----+-------+--------+---------+



#### Airlines data dictionary

In [17]:
airlines_df.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)



In [18]:
# DTypes in pd formatting
pd.DataFrame(airlines_df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,IATA_CODE,string
1,AIRLINE,string


In [19]:
airlines_df.show(20)

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       NK|    Spirit Air Lines|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
+---------+--------------------+



In [20]:
# Dataframe shape
print("Dataframe row count: ", airlines_df.count())
print("Dataframe columns count: ", len(airlines_df.columns))

Dataframe row count:  14
Dataframe columns count:  2


In [21]:
# Column statistics
airlines_df.describe().toPandas()

,summary,IATA_CODE,AIRLINE
0,count,14,14
1,mean,None,None
2,stddev,None,None
3,min,AA,Alaska Airlines Inc.
4,max,WN,Virgin America


In [22]:
missing_values_count(airlines_df)

+---------+-------+
|IATA_CODE|AIRLINE|
+---------+-------+
|        0|      0|
+---------+-------+



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.



In [23]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [24]:
# fig = plt.figure(figsize=(25, 15))
# st = fig.suptitle("Distribution of Features", fontsize=50, verticalalignment="center")
# for col, num in zip(df.toPandas().describe().columns, range(1,11)):
#     ax = fig.add_subplot(3,4, num)
#     ax.hist(df.toPandas[col])
#     plt.grid(False)
#     plt.xticks(ratation=45, fontsize=20)
#     plt.yticks(fontsize=15)
#     plt.title(col.upper(), fontsize=20)
    
# plt.tight_layout()
# st.set_y(0.95)
# fig.subplots_adjust(top=0.85, hspace=0.4)
# plt.show()


#### Cleaning Steps
Document steps necessary to clean the data

- converting to integers
- converting dates to datetime
- ~~converting time to time~~
- convert DIVERTED CANCELLED to Boolean

In [25]:
# Converting to integers 

int_cols = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_DELAY', 'TAXI_OUT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'TAXI_IN', 
            'ARRIVAL_DELAY', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
for c in int_cols :
    df = df.withColumn(c, F.col(c).cast("int"))

In [26]:
# Converting date

df = df.withColumn('DATE', F.make_date('YEAR', 'MONTH', 'DAY'))

In [27]:
# Converting time
# df1 = df.withColumn('SCHEDULED_DEPARTURE', F.make_timestamp(hour=F.substring('SCHEDULED_DEPARTURE', 1, 2),min=F.substring('SCHEDULED_DEPARTURE', 3, 2)))
# Converting time
# df1 = df.withColumn('SCHEDULED_DEPARTURE', F.to_timestamp('SCHEDULED_DEPARTURE', format='HHmm'))

In [28]:
# Converting booleans

df = df.withColumn('DIVERTED', F.col('DIVERTED').cast('boolean'))
df = df.withColumn('CANCELLED', F.col('CANCELLED').cast('boolean'))

In [29]:
df.show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|      DATE|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-

#### Post cleaning statistics

In [30]:
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = 

In [31]:
# DTypes in pd formatting
pd.DataFrame(df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,YEAR,int
1,MONTH,int
2,DAY,int
3,DAY_OF_WEEK,int
4,AIRLINE,string
5,FLIGHT_NUMBER,string
6,TAIL_NUMBER,string
7,ORIGIN_AIRPORT,string
8,DESTINATION_AIRPORT,string
9,SCHEDULED_DEPARTURE,string


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

##### FLIGHTS table fact
- ID PK serial PK
- FLIGHT_ID serial FK SCHEDULE
- TAIL_NUMBER
- DEPARTURE_TIME
- DEPARTURE_DELAY int
- TAXI_OUT int
- WHEELS_OFF
- ELAPSED_TIME
- AIR_TIME
- DISTANCE
- WHEELS_ON
- TAXI_IN
- ARRIVAL_TIME
- ARRIVAL_DELAY
- DIVERTED
- CANCELLED
- CANCELLATION_REASON
- AIR_SYSTEM_DELAY
- SECURITY_DELAY
- AIRLINE_DELAY
- LATE_AIRCRAFT_DELAY
- DATE


##### SCHEDULE table dim
- FLIGHT_ID serial PK composite key AIRLINE FLIGHT NUMBER SCHEDULED DEPARTURE SCHEDULED TIME
- AIRLINE str
- FLIGHT_NUMBER str
- ORIGIN_AIRPOR
- DESTINATION_AIRPORT
- SCHEDULED_DEPARTURE
- SCHEDULED_TIME
- SCHEDULED_ARRIVAL

##### AIRPORTS table dim
- IATA_CODE serial PK
- AIRPORT str
- CITY_ID serial FK CITIES
- LATITUDE
- LONGITUDE

##### AIRLINES table dim
- IATA_CODE serial PK
- AIRLINE str

##### CITIES table dim
- ID serial PK
- CITY str UNIQUE
- STATE str
- COUNTRY str

#### TIME table dim
- DATE date PK
- YEAR
- MONTH
- DAY
- DAY_OF_WEEK

![Image](Img\data-org-chart.png)

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Create a new column ID for storing primary key of FLIGHTS fact table by generating serial for each row in df
2. Create a new columns FLIGHTS_ID for storing primary key of SCHEDULE dim table by creating composite key by concating AIRLINE, FLIGHT_NUMBER, SCHEDULED_ARRIVAL, SCHEDULED_DEPARTURE
3. Create a FLIGHTS fact table by selecting columns into flights_df dataframe
4. Save FLIGHTS table in PARQUET format partitioning by YEAR > MONTH > DAY
5. Create SCHEDULE dim table by selecting appropriate columns with unique FLIGHT_IDs into schedule_df dataframe
6. Save SCHEDULE table in PARQUET format
7. Create CITY dim table by selecting appropriate columns with unique composite key CITY, STATE, COUNTRY and generate CITY_ID
8. Save CITY dim table as parquet partitioning by COUNTRY, STATE
9. Create AIRPORTS dim table by selecting appropriate columns 
10. Save AIRPORTS dim table in PARQUET format partitioning by first letter of IATA_CODE
11. Create AIRLINES dim table by slecting appropriate columns paritioning by first letter of IATA_CODE
12. Create TIME dim table by selecting date columns

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [129]:
# Generate ID columns
df = df.withColumn('ID', F.monotonically_increasing_id())
df = df.withColumn('FLIGHT_ID', F.concat(col('AIRLINE'), col('FLIGHT_NUMBER'), col('SCHEDULED_DEPARTURE'), col('SCHEDULED_ARRIVAL')))

# FLIGHTS fact table
flight_cols = ['ID', 'FLIGHT_ID', 'TAIL_NUMBER', 'DATE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 
                  'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY']
flights_table = df.select(*flight_cols)
flights_table.alias('flights')
# flights_table.write.mode('overwrite').partitionBy('DATE').parquet("Output/flights_table.parquet")

# SCHEDULE dim table
schedule_cols = ['FLIGHT_ID', 'AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'SCHEDULED_ARRIVAL']
schedule_table = df.select(*schedule_cols).distinct().alias('schedule')
# schedule_table.write.mode('overwrite').parquet("Output/schedule_table.parquet")

# AIRLINES dim table
airlines_table = airlines_df.dropDuplicates(['IATA_CODE']).alias('airlines')
# airlines_table.write.mode('overwrite').parquet("Output/airlines_table.parquet")

# w = Window().partitionBy('CITY', 'STATE', 'COUNTRY')
cities_table = airports_df.select('CITY', 'STATE', 'COUNTRY').distinct()
cities_table = cities_table.withColumn('ID', F.monotonically_increasing_id()).select('ID', 'CITY', 'STATE', 'COUNTRY').alias('cities')
# cities_table.write.mode('overwrite').partitionBy('COUNTRY', 'STATE').parquet('Output/cities_table.parquet')

airports_table = airports_df.join(cities_table, on=['CITY', 'STATE', 'COUNTRY'], how='left').drop('CITY', 'STATE', 'COUNTRY').withColumnRenamed('ID', 'CITY_ID').dropDuplicates(['IATA_CODE']).alias('airports')
# airports_table.write.mode('overwrite').parquet('Output/airports_table.parquet')

# TIME dim table
time_cols = ['DATE', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK']
time_table = df.select(*time_cols).distinct().orderBy(col('DATE')).alias('time')
# time_table.write.mode('overwrite').partitionBy('YEAR','MONTH').parquet('Output/time_table.parquet')

It was noticed 58393 records of SCHEDULE table have varying ORIGIN_AIRPORT or DESTINATION_AIRPORT despite having identical values in other fields. 

Inconsistencies in the schedule table are noticed. Sometimes the flight with identical airline, flight, number, scheduled departure, scheduled arrival has inconsitency in origin or destination airports, scheduled time and distance. 

SCHEDULE.SCHEDULED_TIME col is inconsistend with scheduled_departure and scheduld_arrival

In [94]:
schedule_table.groupBy('FLIGHT_ID','SCHEDULED_TIME').agg(F.first('SCHEDULED_DEPARTURE'),F.first('SCHEDULED_ARRIVAL')).show()

+--------------+--------------+--------------------------+------------------------+
|     FLIGHT_ID|SCHEDULED_TIME|first(SCHEDULED_DEPARTURE)|first(SCHEDULED_ARRIVAL)|
+--------------+--------------+--------------------------+------------------------+
|AA100011551521|           146|                      1155|                    1521|
|AA100011551527|           152|                      1155|                    1527|
|AA100016071755|           168|                      1607|                    1755|
|AA100016151810|           175|                      1615|                    1810|
|AA100210301135|            65|                      1030|                    1135|
|AA100212251340|            75|                      1225|                    1340|
|AA100220002221|           201|                      2000|                    2221|
|AA100220052224|           199|                      2005|                    2224|
|AA100220092233|           204|                      2009|                  

In [101]:
schedule_table.filter(F.expr('SCHEDULED_TIME != SCHEDULED_ARRIVAL - SCHEDULED_DEPARTURE')).show()

+--------------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+
|     FLIGHT_ID|AIRLINE|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|SCHEDULED_TIME|SCHEDULED_ARRIVAL|
+--------------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+
|OO738906070724|     OO|         7389|           GCC|                RKS|               0607|            77|             0724|
| NK16307311032|     NK|          163|           DFW|                ATL|               0731|           121|             1032|
|AA233207400955|     AA|         2332|           DFW|                ORD|               0740|           135|             0955|
| UA21107430932|     UA|          211|           FLL|                IAH|               0743|           169|             0932|
|OO523707451041|     OO|         5237|           MKE|                IAH|               0745|           176|   

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [33]:
def test_is_unique():

    def check_unique(df, column):
        check = df.select(column).count() == df.select(column).distinct().count()
        print(f"{df} {column} column unique values test: {check}")
        return check
    
    check_unique(flights_table, 'ID')
    check_unique(schedule_table, 'FLIGHT_ID')
    check_unique(airports_table, 'IATA_CODE')
    check_unique(airlines_table, 'IATA_CODE')
    check_unique(cities_table, 'ID')
    check_unique(time_table, 'DATE')    


In [54]:
def test_datatype():
    
    def check_datatype(df, column, dtype):
        check = dict(df.dtypes)[column] == dtype
        print(f"{df} {column} column datatype {dtype} test: {check}")
        return check

    check_datatype(flights_table, 'DATE', 'date')

In [35]:
def test_count():

    def check_count(df, n):
        check = df.size() > n
        print(f"{df} count test: {check}")
        return check    
    
    check_count(flights_table, 500000)
    check_count(airlines_table, 10)


In [55]:
test_is_unique()
test_datatype()
test_count()

DataFrame[ID: bigint, FLIGHT_ID: string, TAIL_NUMBER: string, DATE: date, DEPARTURE_TIME: string, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: string, ELAPSED_TIME: int, AIR_TIME: int, WHEELS_ON: string, TAXI_IN: int, ARRIVAL_TIME: string, ARRIVAL_DELAY: int, DIVERTED: boolean, CANCELLED: boolean, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int] ID column unique values test: True
DataFrame[FLIGHT_ID: string, AIRLINE: string, FLIGHT_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: string, SCHEDULED_TIME: int, DISTANCE: int, SCHEDULED_ARRIVAL: string] FLIGHT_ID column unique values test: False
DataFrame[IATA_CODE: string, AIRPORT: string, LATITUDE: double, LONGITUDE: double, CITY_ID: bigint] IATA_CODE column unique values test: True
DataFrame[IATA_CODE: string, AIRLINE: string] IATA_CODE column unique values test: True
DataFrame[ID: bigint, CITY: string, STATE: string, COUNT

AttributeError: 'DataFrame' object has no attribute 'size'

In [79]:
schedule_table.count()

543207

In [78]:
schedule_table.orderBy('FLIGHT_ID').show()

+--------------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+
|     FLIGHT_ID|AIRLINE|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|SCHEDULED_TIME|SCHEDULED_ARRIVAL|
+--------------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+
|AA100001400603|     AA|         1000|           DFW|                LGA|               0140|           203|             0603|
|AA100011551516|     AA|         1000|           DFW|                TPA|               1155|           141|             1516|
|AA100011551521|     AA|         1000|         11298|              15304|               1155|           146|             1521|
|AA100011551521|     AA|         1000|           DFW|                TPA|               1155|           146|             1521|
|AA100011551527|     AA|         1000|           DFW|                TPA|               1155|           152|   

In [130]:
df.groupBy('FLIGHT_ID', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT').count().orderBy('FLIGHT_ID', F.desc('COUNT')).show(100)

+--------------+--------------+-------------------+-----+
|     FLIGHT_ID|ORIGIN_AIRPORT|DESTINATION_AIRPORT|count|
+--------------+--------------+-------------------+-----+
|AA100001400603|           DFW|                LGA|    1|
|AA100011551516|           DFW|                TPA|   56|
|AA100011551521|         11298|              15304|   31|
|AA100011551521|           DFW|                TPA|   26|
|AA100011551527|           DFW|                TPA|   97|
|AA100016051800|           TPA|                DFW|   54|
|AA100016071755|         15304|              11298|   31|
|AA100016071755|           TPA|                DFW|   26|
|AA100016121750|           TPA|                DFW|   97|
|AA100016151810|           TPA|                DFW|    2|
|AA100105000841|           BOS|                DFW|   15|
|AA100118251945|           DFW|                TUS|    5|
|AA100118301854|           DFW|                TUS|   21|
|AA100118301954|           DFW|                TUS|   61|
|AA10020705094

In [137]:
schedule_table.groupBy('FLIGHT_ID').count().orderBy(F.desc('count')).show()

+--------------+-----+
|     FLIGHT_ID|count|
+--------------+-----+
|AA200212001327|    4|
|WN456916001725|    4|
|   AA907001016|    3|
|  WN1709201020|    3|
|WN358918151925|    3|
|AA129212251335|    3|
|  WN2510501155|    3|
|AA197010201218|    3|
| WN32610401250|    3|
| WN51610451800|    3|
| AS14322550140|    3|
| AA82608151005|    3|
| WN28116251730|    3|
|   WN408350940|    3|
|WN198214501555|    3|
|B6163519212039|    3|
|OO648209091057|    3|
|WN259606350800|    3|
| WN68310101220|    3|
|WN205106300750|    3|
+--------------+-----+
only showing top 20 rows



In [46]:
df.groupBy('FLIGHT_ID').agg(F.concat('ORIGIN_AIRPORT')).show()

AnalysisException: expression 'ORIGIN_AIRPORT' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;
Aggregate [FLIGHT_ID#2357], [FLIGHT_ID#2357, concat(ORIGIN_AIRPORT#24) AS concat(ORIGIN_AIRPORT)#2885]
+- Project [YEAR#1520, MONTH#1552, DAY#1584, DAY_OF_WEEK#1616, AIRLINE#21, FLIGHT_NUMBER#22, TAIL_NUMBER#23, ORIGIN_AIRPORT#24, DESTINATION_AIRPORT#25, SCHEDULED_DEPARTURE#26, DEPARTURE_TIME#27, DEPARTURE_DELAY#1648, TAXI_OUT#1680, WHEELS_OFF#30, SCHEDULED_TIME#1712, ELAPSED_TIME#1744, AIR_TIME#1776, DISTANCE#1808, WHEELS_ON#35, TAXI_IN#1840, SCHEDULED_ARRIVAL#37, ARRIVAL_TIME#38, ARRIVAL_DELAY#1872, DIVERTED#2097, ... 10 more fields]
   +- Project [YEAR#1520, MONTH#1552, DAY#1584, DAY_OF_WEEK#1616, AIRLINE#21, FLIGHT_NUMBER#22, TAIL_NUMBER#23, ORIGIN_AIRPORT#24, DESTINATION_AIRPORT#25, SCHEDULED_DEPARTURE#26, DEPARTURE_TIME#27, DEPARTURE_DELAY#1648, TAXI_OUT#1680, WHEELS_OFF#30, 

In [123]:
schedule_table = schedule_table.groupBy('FLIGHT_ID','AIRLINE','FLIGHT_NUMBER','SCHEDULED_DEPARTURE','SCHEDULED_TIME','SCHEDULED_ARRIVAL').agg(
        F.concat_ws("",F.collect_list("ORIGIN_AIRPORT")).alias("ORIGIN_AIRPORT"),
        F.concat_ws("",F.collect_list("DESTINATION_AIRPORT")).alias("DESTINATION_AIRPORT")
        ).select(schedule_cols)

In [72]:
schedule_table.select('ORIGIN_AIRPORT').filter(col("ORIGIN_AIRPORT").rlike("^[0-9]*$")).count()

47710

In [124]:
schedule_table.groupBy('FLIGHT_ID').count().orderBy('COUNT', ascending=False).show()

+--------------+-----+
|     FLIGHT_ID|count|
+--------------+-----+
|WN166920202255|    2|
|OO454708370950|    2|
| UA39819102041|    2|
| UA64106000732|    2|
|UA125218562110|    2|
|WN140410101240|    2|
| WN43512351400|    2|
|EV420315171655|    2|
|UA104719062213|    2|
| VX26019502120|    2|
|UA112306470930|    2|
|UA153010351315|    2|
|UA158419152106|    2|
|UA167412361515|    2|
|WN115615451645|    2|
| WN32610401250|    2|
| NK32421302340|    2|
| NK41622150107|    2|
|WN578515351650|    2|
|WN190612401500|    2|
+--------------+-----+
only showing top 20 rows



In [89]:
schedule_table.groupBy('FLIGHT_ID').count().orderBy('count', ascending=False).filter(col('count') > 1).agg(F.sum('count')).collect()

[Row(sum(count)=58393)]

In [ ]:
UA39819102041
UA64106000732
AA200212001327
WN456916001725

In [143]:
schedule_table.where(col('FLIGHT_ID') == 'AA100011551521').show()

+--------------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+
|     FLIGHT_ID|AIRLINE|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|SCHEDULED_TIME|SCHEDULED_ARRIVAL|
+--------------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+
|AA100011551521|     AA|         1000|         11298|              15304|               1155|           146|             1521|
|AA100011551521|     AA|         1000|           DFW|                TPA|               1155|           146|             1521|
+--------------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+



In [144]:
df.where(col('FLIGHT_ID') == 'AA100011551521').show(100)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+-----------+--------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|      DATE|         ID|     FLIGHT_ID|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+-------------

In [90]:
schedule_table.where(F.length('ORIGIN_AIRPORT') > 10).orderBy('FLIGHT_ID').show(50)

+---------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+
|FLIGHT_ID|AIRLINE|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|SCHEDULED_TIME|SCHEDULED_ARRIVAL|
+---------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+
+---------+-------+-------------+--------------+-------------------+-------------------+--------------+-----------------+



In [154]:
schedule_table.groupBy('FLIGHT_ID','ORIGIN_AIRPORT','DESTINATION_AIRPORT').count().filter(col('ORIGIN_AIRPORT').rlike("^[0-9]*$")).show()

+--------------+--------------+-------------------+-----+
|     FLIGHT_ID|ORIGIN_AIRPORT|DESTINATION_AIRPORT|count|
+--------------+--------------+-------------------+-----+
|WN440908351005|         13796|              14679|    1|
|MQ348416081741|         11140|              11298|    1|
|  WN4716351725|         13851|              11259|    1|
|OO740213471503|         11587|              11433|    1|
|OO482513101417|         11898|              13487|    1|
|DL179222300427|         14679|              10397|    1|
| WN26613301540|         13232|              11042|    1|
|  B64910261300|         10721|              14685|    1|
| B629810041238|         13204|              12953|    1|
|B6124822450636|         12889|              12478|    1|
| B649011311253|         11278|              10721|    1|
|UA126819302331|         12266|              10821|    1|
|DL155210091238|         14057|              12892|    1|
|OO651208251149|         12892|              11109|    1|
|DL25660635121

In [ ]:
test_is_unique()

df ID column unique values test: True
df FLIGHT_ID column unique values test: False
df IATA_CODE column unique values test: True
df IATA_CODE column unique values test: True
df ID column unique values test: True
df DATE column unique values test: True


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.